In [1]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
import pandas as pd
import statistics
import time
from tqdm import tqdm
import seaborn as sns
%matplotlib qt

In [2]:
# Initializing the model

xls = pd.ExcelFile('Data_2D_IMP.xlsx')  # parsing Reservoir data
Reservoir_data = pd.read_excel(xls, 'Sheet1') 
print(Reservoir_data)
print("\n")
phi=np.float64(Reservoir_data['Value'][0]) # defining porosity
L=np.float64(Reservoir_data['Value'][1])  # defining reservoir length
W=np.float64(Reservoir_data['Value'][1])  # defining reservoir width
H=np.float64(Reservoir_data['Value'][2])  # defining bed thickness
mu=np.float64(Reservoir_data['Value'][3])  # defining oil viscosity
Ct=np.float64(Reservoir_data['Value'][4])  # defining total compressibility
Bo=np.float64(Reservoir_data['Value'][5])  # defining oil FVF
Del_t=np.float64(Reservoir_data['Value'][6]) # defining time-step value
Nt=int(Reservoir_data['Value'][7])  # defining no. of time steps
N=int(Reservoir_data['Value'][8])  # defining no. of grid blocks in each direction for a regular square reservoir gridding
Del_x=L/N  # estimated block length
print("\nSize of Block: ", Del_x, "ft")
Pi=float(Reservoir_data['Value'][9])  # initial reservoir pressure
qsc=float(Reservoir_data['Value'][10])  # surface oil injection/withdrawal rate as source/sink

# Defining Permeability Heterogeneity

k_ = pd.read_excel('Permeability_Heterogeneity_2D_IMP.xlsx')

k = np.zeros((N,N)) # permeability for the blocks
for i in range(N):
    for j in range(N):
        k[i,j] = np.float64(k_[j][i])
        
ax = sns.heatmap( k , linewidth = 0.001 , cmap = 'coolwarm' )
  
plt.title( "2D Permeability Heterogeneity" )
plt.show()

                                             Property Annotation        Value
0                               Porosity in fraction:        phi     0.500000
1                             Reservoir Length in ft:          L  1000.000000
2                             Reservoir Height in ft:          H    10.000000
3                                    Viscosity in cp:         mu     0.800000
4                    Total Compressibility in psi^-1:         Ct     0.000005
5            Fluid Formation Volume Factor in rb/stb:         Bo     1.100000
6                                  Time Step in days:      Del_t     0.010000
7                               Number of Time Steps:         Nt   150.000000
8                Number of Blocks in both directions:          N    41.000000
9                   Reservoir Pressure in psi at t=0:         Pi  2000.000000
10  Surface flow rate at center of reservoir in st...        qsc   100.000000



Size of Block:  24.390243902439025 ft


In [3]:
# Initializing Implicit Formulation

print("\nIC \n at t=0, Reservoir Pressure is ", Pi, " psi \nBCs \n Surface production rate at the centre block, i.e. (",(N+1)/2,",",(N+1)/2,") is ", qsc, "stb/day \n at the reservoir boundaries (NEUMANN BC), NO  FLOW is there i.e., (Del_P/Del_x)=0\n")
Psc=5.615*Bo*Del_t*qsc/H/(Del_x**2)/phi/Ct

k50=statistics.mean(k.flatten())
k84=statistics.stdev(k.flatten())
V=k84/k50  # Dykstra-Parsons Hetergeneity coefficient
print("\nEstimation of Dykstra-Parsons Heterogeneity Coefficient:\nFrom the given Permeability data, K50 = ",k50," md and K84.1 = ",k84, " md \nSo, V = (k50-k84)/k50 = ",V)

print("\n")

# Estimation of Interblock k and eta values along x- and y-directions
print("Estimation of InterBlock k and eta Values:")
kxib=np.zeros((N,N-1))
for i in range(N):
    for j in range(N-1):
        kxib[i,j]=np.float64(2/((1/k[i,j])+(1/k[i,j+1]))) # Harmonic Average for flow media in series # x-direction
kyib=np.zeros((N-1,N))
for i in range(N-1):
    for j in range(N):
        kyib[i,j]=np.float64(2/((1/k[i,j])+(1/k[i+1,j]))) # y-direction
print("Interblock k-")
print("in x-direction-")
print(kxib)
print("in y-direction-")
print(kyib)
print("Interblock eta-")
etax=0.00633*kxib*Del_t/phi/mu/Ct/(Del_x**2) # x-direction
etay=0.00633*kyib*Del_t/phi/mu/Ct/(Del_x**2) # y-direction
print("in x-direction-")
print(etax)
print("in y-direction-")
print(etay)


print("\n")


# Estimation of Penta-Diagonal Coefficient Matrix for Implicit formulation
etamat=np.zeros((N**2,N**2))
for i in range(0,N):
    for j in range(0,N):
        if j==i-1:
            etamat[i,j]=-etax[0,j]
            
        elif j==i and i!=0 and i!=N-1:
            etamat[i,j]=1+etax[0,i-1]+etax[0,i]+etay[0,i]

        elif j==i+1:
            etamat[i,j]=-etax[0,i]
            
etamat[0,0]=1+etax[0,0]+etay[0,0]
etamat[N-1,N-1]=1+etax[0,N-2]+etay[0,N-1]
        

for i in range((N**2)-N,N**2):
    for j in range((N**2)-N,N**2):
        if j==i-1:
            etamat[i,j]=-etax[N-1,j-((N**2)-N)]
            
        elif j==i and i!=((N**2)-N) and i!=(N**2-1):
            etamat[i,j]=1+etax[N-1,i-1-((N**2)-N)]+etax[N-1,i-((N**2)-N)]+etay[N-2,i-((N**2)-N)]

        elif j==i+1:
            etamat[i,j]=-etax[N-1,i-((N**2)-N)]
            
etamat[((N**2)-N),((N**2)-N)]=1+etax[N-1,0]+etay[N-2,0]
etamat[(N**2-1),(N**2-1)]=1+etax[N-1,N-2]+etay[N-2,N-1]

    
for g in range(1,N-1):
    for i in range(g*N,(g+1)*N):
        for j in range(g*N,(g+1)*N):
            if j==i-1:
                etamat[i,j]=-etax[g,j-g*N]
            
            elif j==i and i!=(g*N) and i!=((g+1)*N-1):
                etamat[i,j]=1+etax[g,i-1-g*N]+etax[g,i-g*N]+etay[g-1,i-g*N]+etay[g,i-g*N]

            elif j==i+1:
                etamat[i,j]=-etax[g,i-g*N]
            
    etamat[(g*N),(g*N)]=1+etax[g,0]+etay[g-1,0]+etay[g,0]
    etamat[((g+1)*N-1),((g+1)*N-1)]=1+etax[g,N-2]+etay[g-1,N-1]+etay[g,N-1]


for g in range(1,N):
    for i in range(g*N,(g+1)*N):
        etamat[i,i-N]=-etay[g-1,i-g*N]
        etamat[i-N,i]=-etay[g-1,i-g*N]


print("Penta-diagonal Matrix or eta-matrix is estimated as: \n"+str(etamat))
pendiag=pd.DataFrame(etamat)




IC 
 at t=0, Reservoir Pressure is  2000.0  psi 
BCs 
 Surface production rate at the centre block, i.e. ( 21.0 , 21.0 ) is  100.0 stb/day 
 at the reservoir boundaries (NEUMANN BC), NO  FLOW is there i.e., (Del_P/Del_x)=0


Estimation of Dykstra-Parsons Heterogeneity Coefficient:
From the given Permeability data, K50 =  75.1060952527097  md and K84.1 =  39.01464676413915  md 
So, V = (k50-k84)/k50 =  0.5194604596719675


Estimation of InterBlock k and eta Values:
Interblock k-
in x-direction-
[[48.79253674 46.39386995 49.42556508 ... 37.2386197  28.70178024
  10.9688025 ]
 [50.93833133 48.66732843 52.53593729 ... 39.03387897 18.70518584
   2.91802719]
 [61.31057125 63.08378533 63.39653075 ... 31.58228929 21.80721565
   7.53123794]
 ...
 [55.76288258 63.66664012 66.50051027 ... 15.65268175 22.65803086
  37.70604042]
 [54.04905924 63.00313012 67.14215745 ...  7.63805403 29.77312973
  36.1453682 ]
 [60.56115639 62.88668045 62.9735251  ... 33.64341341 33.61317212
  34.3597776 ]]
in y-dir

In [4]:
# Running the simulation model via Implicit Formulation
etainv=np.linalg.inv(etamat)

P_=np.full((Nt+1,N**2),Pi,dtype=np.float64)
for i in range(1,Nt+1):
    B=P_[i-1,:]
    F=np.zeros((N**2))
    F[(25-1)*N+8-1]=-Psc  # Putting the wells at the desired cell-address
    F[(35-1)*N+32-1]=-Psc
    F[(15-1)*N+20-1]=-Psc
    
    
    P_[i,:]=np.dot(etainv,B+F)

P=np.full((Nt+1,N,N),0,dtype=np.float64)
for i in range(Nt+1):
    P[i]=P_[i].reshape(N,N).transpose()

    
print("\n")
m=[]
for i in range(N):
    m.append((i+0.5)*Del_x)
plt.figure(figsize=(8,8))
k=plt.axes(projection='3d')
X, Y = np.meshgrid(m, m)
k.plot_wireframe(X, Y,P[math.floor(Nt/10),:,:],color='r')
k.plot_wireframe(X, Y,P[math.floor(Nt/2),:,:],color='y')
k.plot_wireframe(X, Y,P[10*math.floor(Nt/10),:,:],color='m')
k.set_xlabel('Reservoir Length in ft')
k.set_ylabel('Reservoir Width in ft')
k.set_zlabel('Pressure in psi')
print('Red => Time Step= '+str(math.floor(Nt/10)))
print('Yellow => Time Step= '+str(math.floor(Nt/2)))
print('Magenta => Time Step= '+str(10*math.floor(Nt/10)))
plt.grid()
plt.show()



Red => Time Step= 15
Yellow => Time Step= 75
Magenta => Time Step= 150


In [5]:
# Dynamic Heatmap of the pressure simulation with 20 time levels
%matplotlib qt
Pn=np.full(((Nt//19),N,N),0,dtype=np.float64)
for l in range(0,(Nt//19)):
    for i in range(0,N):
        for j in range(0,N):
            if l==0:
                Pn[l,i,j]=P[1,i,j]
            elif l==1:
                Pn[l,i,j]=P[5*l,i,j]
            else:
                Pn[l,i,j]=P[8*(l-1),i,j]


fig=plt.figure(figsize=(8,8))
plt.style.use('default')
for i in tqdm(range(0,(Nt//19))):
    time.sleep(1.25)
    plt.imshow(Pn[i])
    fig.canvas.draw()
    fig.canvas.flush_events()

100%|██████████| 7/7 [00:09<00:00,  1.34s/it]


In [ ]:
ax = sns.heatmap( etamat , linewidth = 0.0001 , cmap = 'coolwarm' )
  
plt.title( "Penta-Diagonal Coefficient Matrix" )
plt.show()

In [5]:
ax = sns.heatmap( etainv , linewidth = 0.0001 , cmap = 'coolwarm' )
  
plt.title( "Penta-Diagonal Coefficient Matrix" )
plt.show()